In [86]:
import pandas as pd
import geopandas as gpd
import altair as alt

In [87]:
#all_sheets = pd.read_excel('https://docs.google.com/spreadsheets/d/1yZv9w9zRKwrGTaR-YzmAqMefw4wMlaXocejdxZaTs6w/export?format=xlsx', sheet_name=None)
all_sheets = pd.read_excel('https://docs.google.com/spreadsheets/d/1wQVypefm946ch4XDp37uZ-wartW4V7ILdg-qYiDXUHM/export?format=xlsx', sheet_name=None)

In [88]:
print(all_sheets.keys())
df = all_sheets['Jan31_2pm']
df.head()

odict_keys(['Feb03_940pm', 'Feb03_1230pm', 'Feb02_9PM', 'Feb02_745pm', 'Feb02_5am', 'Feb01_11pm', 'Feb01_6pm', 'Feb01_10am', 'Jan31_7pm', 'Jan31_2pm', 'Jan30_930pm', 'Jan30_11am', 'Jan29_9pm', 'Jan29_230pm', 'Jan29_130pm', 'Jan28_11pm', 'Jan28_6pm', 'Jan28_1pm', 'Jan27_830pm', 'Jan27_7pm', 'Jan27_9am', 'Jan26_11pm', 'Jan26_11am', 'Jan25_10pm', 'Jan25_12pm', 'Jan25_12am', 'Jan24_12pm', 'Jan24_12am', 'Jan23_12pm', 'Jan22_12pm', 'Jan22_12am'])


,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,Hubei,Mainland China,2020-01-31 14:00:00,5806,204.0,141.0
1,Zhejiang,Mainland China,2020-01-31 14:00:00,538,NaN,14.0
2,Guangdong,Mainland China,2020-01-31 14:00:00,436,NaN,11.0
3,Henan,Mainland China,2020-01-31 14:00:00,352,2.0,3.0
4,Hunan,Mainland China,2020-01-31 14:00:00,332,NaN,2.0


In [89]:
selection_sheets = list(all_sheets.keys())#[::3]
print(selection_sheets)

['Feb03_940pm', 'Feb03_1230pm', 'Feb02_9PM', 'Feb02_745pm', 'Feb02_5am', 'Feb01_11pm', 'Feb01_6pm', 'Feb01_10am', 'Jan31_7pm', 'Jan31_2pm', 'Jan30_930pm', 'Jan30_11am', 'Jan29_9pm', 'Jan29_230pm', 'Jan29_130pm', 'Jan28_11pm', 'Jan28_6pm', 'Jan28_1pm', 'Jan27_830pm', 'Jan27_7pm', 'Jan27_9am', 'Jan26_11pm', 'Jan26_11am', 'Jan25_10pm', 'Jan25_12pm', 'Jan25_12am', 'Jan24_12pm', 'Jan24_12am', 'Jan23_12pm', 'Jan22_12pm', 'Jan22_12am']


In [90]:
#selection_sheets = ['Jan31_7pm', 'Jan30_930pm', 'Jan29_230pm', 'Jan28_11pm', 'Jan27_7pm', 'Jan26_11pm', 'Jan25_10pm', 'Jan24_12pm', 'Jan23_12pm', 'Jan22_12pm']

In [91]:
df_all = pd.DataFrame()
for sheet in selection_sheets:
    df = all_sheets[sheet]
    df.rename(columns={'Last Update':'Date'}, inplace=True)
    df.rename(columns={'Date last updated':'Date'}, inplace=True) 
    df.rename(columns={'Last Update (UTC)':'Date'}, inplace=True)     
    df_all = pd.concat((df_all, df), axis=0, sort=False)

In [92]:
df_all.replace(
    to_replace=['Ningxia', 'Inner Mongolia', 'Xinjiang', 'Tibet'], 
    value=['Nei Mongol', 'Ningxia Hui','Xinjiang Uygur', 'Xizang'],
    inplace=True
)
list_of_prov = []
for prov in df_all['Province/State'].unique():
    df_prov = df_all[df_all['Province/State'] == prov]
    df_prov = df_prov.fillna(method='ffill')
    list_of_prov.append(df_prov)
df_all = pd.concat(list_of_prov)

In [93]:
df_slice = df_all[['Province/State', 'Date', 'Confirmed', 'Deaths', 'Recovered']]
df_slice['Date'] = pd.to_datetime(df_slice['Date'], 'coerce')
df_slice_agg_1d = df_slice.groupby('Province/State').resample('1d', on='Date').max()

/Users/mattijnvanhoek/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [94]:
df_all = df_slice_agg_1d[['Confirmed', 'Deaths', 'Recovered']].reset_index()
df_all.tail()

,Province/State,Date,Confirmed,Deaths,Recovered
559,Zhejiang,2020-01-31,538.0,0.0,14.0
560,Zhejiang,2020-02-01,599.0,0.0,21.0
561,Zhejiang,2020-02-02,661.0,0.0,32.0
562,Zhejiang,2020-02-03,724.0,0.0,43.0
563,Zhejiang,2020-02-04,829.0,0.0,48.0


In [95]:
url_china_prv = 'https://raw.githubusercontent.com/deldersveld/topojson/master/countries/china/china-provinces.json'
gdf = gpd.read_file(url_china_prv, driver='TopoJSON')
gdf.head()

,id,ISO,NAME_0,ID_1,NAME_1,TYPE_1,ENGTYPE_1,NL_NAME_1,VARNAME_1,geometry
0,None,CHN,China,1,Anhui,Shěng,Province,安徽|安徽,Ānhuī,"POLYGON ((119.63215 31.13933, 119.64444 31.115..."
1,None,CHN,China,2,Beijing,Zhíxiáshì,Municipality,北京|北京,Běijīng,"POLYGON ((117.37966 40.22687, 117.38301 40.208..."
2,None,CHN,China,3,Chongqing,Zhíxiáshì,Municipality,重慶|重庆,Chóngqìng,"POLYGON ((109.57656 31.73312, 109.60255 31.719..."
3,None,CHN,China,4,Fujian,Shěng,Province,福建,Fújiàn,"MULTIPOLYGON (((117.70771 23.78171, 117.72001 ..."
4,None,CHN,China,5,Gansu,Shěng,Province,甘肅|甘肃,Gānsù,"POLYGON ((97.18466 42.77164, 97.39479 42.41457..."


In [96]:
import topojson as tp
t = tp.Topology(gdf)

In [97]:
gdf_topo = t.toposimplify(0.09).topoquantize(1e3).to_gdf()

In [98]:
gdf_sel = gdf_topo[['NAME_1', 'geometry']]

In [99]:
gdf_join = pd.merge(gdf_sel, df_all.fillna(0), how='outer', left_on='NAME_1', right_on='Province/State')

In [100]:
#gdf_join.head()

In [101]:
#gdf_join['Last Update'] = pd.to_datetime(gdf_join['Last Update'], 'coerce')

In [102]:
gdf_join['Date'] = gdf_join['Date'].dt.strftime('%Y-%m-%d')
#gdf_join.head()

In [103]:
gdf_join.rename({'Last Update': 'Update'})
gdf_join_slice = gdf_join[['NAME_1', 'geometry', 'Date', 'Confirmed', 'Deaths', 'Recovered']]
gdf_join_slice.columns = ['Province', 'geometry', 'Date', 'Confirmed', 'Deaths', 'Recovered']

In [104]:
gdf_join_slice.head()

,Province,geometry,Date,Confirmed,Deaths,Recovered
0,Anhui,"POLYGON ((119.63838 31.13065, 119.45455 30.670...",2020-01-21,1.0,0.0,2.0
1,Anhui,"POLYGON ((119.63838 31.13065, 119.45455 30.670...",2020-01-22,1.0,0.0,2.0
2,Anhui,"POLYGON ((119.63838 31.13065, 119.45455 30.670...",2020-01-23,9.0,0.0,2.0
3,Anhui,"POLYGON ((119.63838 31.13065, 119.45455 30.670...",2020-01-24,15.0,0.0,2.0
4,Anhui,"POLYGON ((119.63838 31.13065, 119.45455 30.670...",2020-01-25,60.0,0.0,2.0


In [105]:
column_list = ['Confirmed', 'Deaths', 'Recovered']
row_list = gdf_join_slice['Date'].unique().tolist()

In [106]:
row_list

['2020-01-21',
 '2020-01-22',
 '2020-01-23',
 '2020-01-24',
 '2020-01-25',
 '2020-01-26',
 '2020-01-27',
 '2020-01-28',
 '2020-01-29',
 '2020-01-30',
 '2020-01-31',
 '2020-02-01',
 '2020-02-02',
 '2020-02-03',
 '2020-02-04']

In [107]:
row_list = row_list[1:]
row_list

['2020-01-22',
 '2020-01-23',
 '2020-01-24',
 '2020-01-25',
 '2020-01-26',
 '2020-01-27',
 '2020-01-28',
 '2020-01-29',
 '2020-01-30',
 '2020-01-31',
 '2020-02-01',
 '2020-02-02',
 '2020-02-03',
 '2020-02-04']

In [108]:
#row_list = row_list[:-1]
#row_list[:]

In [109]:
charts_conf = []
charts_death = []
charts_recv = []
for date in row_list[::-1]:
    
    l = alt.Chart(gdf_join_slice).mark_geoshape(stroke='gray').encode(  
        fill=alt.Fill(
            'Confirmed', 
            type='quantitative', 
            scale=alt.Scale(
                type='threshold', 
                domain=[10, 100, 1000, 10000],
                scheme='bluepurple'#['#6ec2de', '#79a7e8','#7e8cf1',  '#7957fb', '#6448dc']#  , 4: '#7e71f9', 
            )
        ),    
        opacity=alt.Opacity('Confirmed', type='quantitative', scale=alt.Scale(type='threshold', domain=[10, 100, 1000, 10000])),    
        tooltip=['Province','Confirmed', 'Deaths', 'Recovered','Date'],
    ).transform_filter(
        alt.FieldEqualPredicate(field='Date', equal=date)
    ).properties(title=date, width=200, height=150)
    charts_conf.append(l)
    
    m = alt.Chart(gdf_join_slice).mark_geoshape(stroke='gray').encode(  
        fill=alt.Fill(
            'Deaths', 
            type='quantitative', 
            scale=alt.Scale(
                type='threshold', 
                domain=[1,10,100,500],
                scheme='redpurple'
            )),    
        opacity=alt.Opacity('Deaths', type='quantitative', scale=alt.Scale(type='threshold', domain=[1,10,100,500])),    
        tooltip=['Province','Confirmed', 'Deaths', 'Recovered','Date'],
    ).transform_filter(
        alt.FieldEqualPredicate(field='Date', equal=date)
    ).properties(title=date, width=200, height=150)
    charts_death.append(m)
    
    r = alt.Chart(gdf_join_slice).mark_geoshape(stroke='gray').encode(  
        fill=alt.Fill(
            'Recovered',
            type='quantitative', 
            scale=alt.Scale(
                type='threshold', 
                domain=[1,10,100,1000],
                scheme='goldgreen'
            )),      
        opacity=alt.Opacity('Recovered', type='quantitative', scale=alt.Scale(type='threshold', domain=[1,10,100,1000])),    
        tooltip=['Province','Confirmed', 'Deaths', 'Recovered', 'Date'],
    ).transform_filter(
        alt.FieldEqualPredicate(field='Date', equal=date)
    ).properties(title=date, width=200, height=150)
    charts_recv.append(r)
    
v_conf = alt.vconcat(*charts_conf).resolve_scale(
    fill='independent',
    opacity='independent'
)
v_death = alt.vconcat(*charts_death).resolve_scale(
    fill='independent',
    opacity='independent'
)
v_recv = alt.vconcat(*charts_recv).resolve_scale(
    fill='independent',
    opacity='independent'
)

h_conc = alt.hconcat(v_conf, v_death, v_recv
)

In [110]:
h_conc

alt.HConcatChart(...)

In [111]:
import hkvportal

In [112]:
dp = hkvportal.Service('https://tsws.hkvservices.nl/mangrove.ws/', '236ebLIeCDdnKbfGe956IvZetIv1')

Dataservice is recognized. https://tsws.hkvservices.nl/mangrove.ws/entry.asmx?WSDL will be used as portal


In [113]:
#dp.create_database(database='2019-nCoV')

In [114]:
dp.update_entry(
    database='2019-nCoV',
    key='maps_conf_deat_reco',
    data=h_conc.to_html(),
    description='small multiples china'
)

entry available at:
https://tsws.hkvservices.nl/mangrove.ws/data.ashx?function=dataportal.db.getdata&parameters={database:'2019-nCoV',key:'maps_conf_deat_reco'}&contentType=SET_BY_USER
